# BASIC CANDIDATE EXECUTION WITH YPROV4ML

Imports needed for BayesianOptimization and candidates execution

In [1]:
import sys, os
from bayesopt import *
sys.path.insert(0, os.path.abspath(os.path.join(os.path.join(os.getcwd()), '..')))
import etl.extractors.provenance_extractor as pe
from typing import Dict, List, Optional
import csv
from datetime import datetime
import torch, torchvision, yprov4ml
import torch.optim as optim
from tqdm import tqdm
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import netCDF4 as nc

### CANDIDATE GENERATION
Here we use the run method to perform Bayesian Optimization and obtain a candidate.

(for detailed explanation of the workflow see the detailed_cand_generation.ipynb notebook)

In [2]:
data_needed = {
    'input': ['DROPOUT', 'BATCH_SIZE', 'EPOCHS', 'LR', 'MODEL_SIZE'],
    'output': ['accuracy', 'emissions']
}

extractor = pe.ProvenanceExtractor('../test/prov_25', data_needed)
inp, out = extractor.extract_all()      # cols are parameters/metrics, rows are runs

bayesopt = BayesianOptimizer(OptimizationConfig(
    data_needed['output'],
    data_needed['input'],
    ['MAX', 'MIN'],
    ground_truth_dim=len(inp),
    n_candidates=1,
    n_restarts=10,
    raw_samples=200,
    optimizers='optimize_acqf',
    acqf='ucb',
    beta=1.5,
    verbose=False
))

data = {
    'parameters': inp,
    'metrics': out
}

# res will be a OptimizationResults instance
res = bayesopt.run(data)

### CANDIDATE EXECUTION
In this block the candidate will be executed with yprov4ml (to see the yprov functionality refer to their documentation). 

NB: for now if the candidate generated has a twin there could be errors in returning the accuracy and emissions, if you notice a 'duplicate candidate' change the _0 to _1 in the _emission call

In [3]:
class Net(nn.Module):
    def __init__(self, model_size, dropout):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        def get_layer_sizes(model_size): 
            if model_size == "small": 
                return 64, 32
            elif model_size == "medium": 
                return 512, 256
            else: 
                return 1024, 256

        l1, l2 = get_layer_sizes(model_size)

        self.fc1 = nn.Linear(12544, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

def train(lr, epochs, batch_size, dropout, model_size):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

    model = Net(model_size, dropout=dropout).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss().to(device)
    scheduler = None

    model.train()

    losses = []
    for _ in range(epochs): 
        for data in tqdm(trainloader):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
            if scheduler is not None:
                scheduler.step()
    yprov4ml.log_carbon_metrics(yprov4ml.Context.TRAINING, step=0)
    return model

def emissions_(expdir_path):
    fp=f'./{expdir_path}/metrics_GR0/emissions_Context.TRAINING_GR0.nc'
    data = nc.Dataset(fp)
    return data["values"][:]

def validate(model, batch_size=128):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

def handle_modelsize(n: float):
    if n <= 3700506.0:      #small
        return 'small'
    elif n <= 9853386.0:    #medium
        return 'medium'
    else:                   #large
        return 'large'

exec_results = []
for candidate in res.candidates:
    candidate[4] = handle_modelsize(candidate[4])   #change to small/medium/large the modelsize
    yprov4ml.start_run(
        prov_user_namespace="www.example.org",
        experiment_name=f"{round(candidate[3], 4)}_{int(candidate[2])}_{int(candidate[1])}_{round(candidate[0], 2)}_{candidate[4]}", 
        provenance_save_dir="../test/prov_candidates_executed",     # change the folder in which you want to save the candidate executed
        save_after_n_logs=100,
        collect_all_processes=False, 
        disable_codecarbon=False, 
        metrics_file_type=yprov4ml.MetricsType.NETCDF,
    )

    yprov4ml.log_param("MODEL_SIZE", candidate[4], yprov4ml.Context.TRAINING)
    yprov4ml.log_param("DROPOUT", candidate[0], yprov4ml.Context.TRAINING)
    yprov4ml.log_param("BATCH_SIZE", candidate[1], yprov4ml.Context.TRAINING)
    yprov4ml.log_param("EPOCHS", candidate[2], yprov4ml.Context.TRAINING)
    yprov4ml.log_param("LR", candidate[3], yprov4ml.Context.TRAINING)

    trained_model = train(candidate[3], int(candidate[2]), int(candidate[1]), candidate[0], candidate[4])
    acc = validate(trained_model, int(candidate[1]))

    yprov4ml.log_param("accuracy", acc, yprov4ml.Context.TESTING)

    yprov4ml.end_run(
        create_graph=False,
        create_svg=False,
        crate_ro_crate=False
    )

    print(f'Accuracy: {100 * acc} %')
    em = emissions_(f"../test/prov_candidates_executed/{round(candidate[3], 4)}_{int(candidate[2])}_{int(candidate[1])}_{round(candidate[0], 2)}_{candidate[4]}_0")[0]
    print(f'Emissions: {em}')
    exec_results.append([acc, em])

[codecarbon WARNING @ 21:23:26] Multiple instances of codecarbon are allowed to run at the same time.
100%|██████████| 323/323 [00:07<00:00, 40.56it/s]


Accuracy: 10.0 %
Emissions: 0.06714221835136414


### SAVE CANDIDATES AND THEIR RESULTS TO A CSV

In [4]:
class CSVResults:
    """class to write to a csv file the results obtained from Bayesian Optimization
    
    Attributes:
        csv_path (str): path for the file to be accessed/created
        parameter_names (List(str)): names of the parameters optimized (for headers)
        metrics_names (List(str)): names of the metrics (for headers)
        metrics_estim_names (List(str)): names of the estimated metrics (for headers)
    """
    def __init__(self, headers: Dict[str, List[str]], path: str):
        self.csv_path = path
        self.parameter_names = headers['parameters']
        self.metrics_names = headers['metrics']
        self.metrics_estim_names = [f'estimated {m}' for m in headers['metrics']]
        self.ensure_exists()

    def build_headers(self):
        headers = ['timestamp', 'candidate_id', 'acq_value', 'optimizer', 'acqf', 'n_restarts', 'raw_samples']

        headers.extend(self.parameter_names)

        for metric in self.metrics_estim_names:
            headers.append(f'{metric} (mean)')
            headers.append(f'{metric} (std)')

        headers.append('execution status')
        headers.extend(self.metrics_names)
        return headers

    def ensure_exists(self):
        if not os.path.exists(self.csv_path):
            with open(self.csv_path, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(self.build_headers())
    
    def log_candidates(self, results: OptimizationResults, config: OptimizationConfig, exec_results: Optional[List[List]]=None):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # shape [n_candidates x n_metrics]
        mean = results.posterior.mean.detach().numpy()
        std = results.posterior.variance.sqrt().detach().numpy()

        rows_to_write = []

        if isinstance(results.acq_values, list):
            acq_val = results.acq_values
        else:
            acq_val = [results.acq_values]*len(results.candidates)

        for i, candidate in enumerate(results.candidates):
            if isinstance(candidate, torch.Tensor):
                candidate = candidate.tolist()

            row = [timestamp, i+1, round(acq_val[i], 6), config.optimizers, config.acqf, config.n_restarts, config.raw_samples]
            for param_val in candidate:
                if isinstance(param_val, float):
                    row.append(round(param_val, 6))
                else:
                    row.append(param_val)

            for m in range(len(self.metrics_estim_names)):
                if m < mean.shape[1]:
                    row.append(round(mean[i][m].item(), 6))
                    row.append(round(std[i][m].item(), 6))
                else:
                    row.extend([None, None])
            
            if exec_results and i < len(exec_results):
                row.append('executed')
                for m in range(len(self.metrics_names)):
                    value = exec_results[i][m]
                    if isinstance(value, float):
                        row.append(round(value, 6))
                    else:
                        row.append(value)
            else:
                row.append('not_executed')
                row.extend([None]*len(self.metrics_names))
            
            rows_to_write.append(row)

        self.append_rows(rows_to_write)

    def append_rows(self, rows: List[List]):
        with open(self.csv_path, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(rows)

In [5]:
csv_saver = CSVResults(
    {'parameters': ['DROPOUT', 'BATCH_SIZE', 'EPOCHS', 'LR', 'MODEL_SIZE'], 
    'metrics': ['accuracy', 'emissions']},
    './candidates_executed.csv')
csv_saver.log_candidates(res, bayesopt.config, exec_results)

### VISUALIZE RESULTS

In the following table we can observe the results obtained by generate, estimate and execute candidates.

NB: the estimated emission is negative because it's a metric to minimize (so we have to consider it as positive)

In [6]:
import pandas as pd

df = pd.read_csv('./candidates_executed.csv')
df

,timestamp,candidate_id,acq_value,optimizer,acqf,n_restarts,raw_samples,DROPOUT,BATCH_SIZE,EPOCHS,LR,MODEL_SIZE,estimated accuracy (mean),estimated accuracy (std),estimated emissions (mean),estimated emissions (std),execution status,accuracy,emissions
0,2026-01-15 21:56:51,1,0.377749,optimize_acqf,ucb,10,200,0.498863,31.004065,10.035223,0.084505,large,0.737886,0.017958,-0.004368,0.000387,executed,0.1,0.067142


### RE-EXECUTE APPENDING CANDIDATE
Now we append the result that we obtained and re-generate candidates with the updated training dataset

NB: if you change model with this type of iterative execution there could be problems 

In [7]:
model_size = 824862.0 if res.candidates[0][4] == 'small' else 6576330.0 if res.candidates[0][4] == 'medium' else 13130442.0
new_data = {
    'parameters': [[res.candidates[0][0], res.candidates[0][1], res.candidates[0][2], res.candidates[0][3], model_size]],
    'metrics': exec_results
}

bayesopt.update_training_set(new_data)
second_res = bayesopt.run()

### SAVE THE NEW CANDIDATE WITHOUT EXECUTION

In [8]:
csv_saver.log_candidates(second_res, bayesopt.config)